##Install and Import the Required Libraries

In [ ]:
# Install all the required libraries

!pip install -U -q pdfplumber tiktoken openai chromaDB sentence-transformers --quiet

In [ ]:
# Import all the required Libraries

import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
import chromadb

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## <font color = magenta> 1. Layer 1 : Embedding Layer

### 1.1 Read, Process, and Chunk the PDF Files

In [ ]:
pdf_path = '/content/drive/MyDrive/upgrad/Course 3 - RAG'

In [ ]:
def check_bboxes(word, table_bbox):
    # Check whether word is inside a table bbox.
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

In [ ]:
def extract_text_from_pdf(pdf_path):
    p = 0
    full_text = []


    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_no = f"Page {p+1}"
            text = page.extract_text()

            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any(
                [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
            lines = []

            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):

                if 'text' in cluster[0]:
                    try:
                        lines.append(' '.join([i['text'] for i in cluster]))
                    except KeyError:
                        pass

                elif 'table' in cluster[0]:
                    lines.append(json.dumps(cluster[0]['table']))


            full_text.append([page_no, " ".join(lines)])
            p +=1

    return full_text

*Now that we have defined the function for extracting the text and tables from a PDF, let's iterate and call this function for all the PDFs in our drive and store them in a list.*

In [ ]:
# Define the directory containing the PDF files
pdf_directory = Path(pdf_path)

# Initialize an empty list to store the extracted texts and document names
data = []

# Loop through all files in the directory
for pdf_path in pdf_directory.glob("*.pdf"):

    # Process the PDF file
    print(f"...Processing {pdf_path.name}")

    # Call the function to extract the text from the PDF
    extracted_text = extract_text_from_pdf(pdf_path)

    # Convert the extracted list to a PDF, and add a column to store document names
    extracted_text_df = pd.DataFrame(extracted_text, columns=['Page No.', 'Page_Text'])
    extracted_text_df['Document Name'] = pdf_path.name

    # Append the extracted text and document name to the list
    data.append(extracted_text_df)

    # Print a message to indicate progress
    print(f"Finished processing {pdf_path.name}")

# Print a message to indicate all PDFs have been processed
print("All PDFs have been processed.")

...Processing Principal-Sample-Life-Insurance-Policy.pdf
Finished processing Principal-Sample-Life-Insurance-Policy.pdf
All PDFs have been processed.


In [ ]:
# Concatenate all the DFs in the list 'data' together
insurance_pdfs_data = pd.concat(data, ignore_index=True)

# All the text are in upper case - lets convert them to lower case
insurance_pdfs_data.Page_Text = insurance_pdfs_data.Page_Text.str.lower()

# Drop the duplicate rows
insurance_pdfs_data.Page_Text = insurance_pdfs_data.Page_Text.apply(lambda x: x.replace('page',''))

# Let's also check the length of all the texts as there might be some empty pages or pages with very few words that we can drop
insurance_pdfs_data['Text_Length'] = insurance_pdfs_data['Page_Text'].apply(lambda x: len(x.split(' ')))

# Retain only the rows with a text length of at least 10
insurance_pdfs_data = insurance_pdfs_data.loc[insurance_pdfs_data['Text_Length'] >= 10]

# Store the metadata for each page in a separate column
insurance_pdfs_data['Metadata'] = insurance_pdfs_data.apply(lambda x: {'Policy_Name': x['Document Name'][:-4], 'Page_No.': x['Page No.'].split(' ')[1]}, axis=1)

In [ ]:
insurance_pdfs_data.head(2)

,Page No.,Page_Text,Document Name,Text_Length,Metadata
0,Page 1,dorothea glause s655 rhode island john doe 01/...,Principal-Sample-Life-Insurance-Policy.pdf,30,{'Policy_Name': 'Principal-Sample-Life-Insuran...
2,Page 3,policy rider group insurance policy no: s655 c...,Principal-Sample-Life-Insurance-Policy.pdf,230,{'Policy_Name': 'Principal-Sample-Life-Insuran...


In [ ]:
insurance_pdfs_data.Page_Text[18]

't he principal has complete discretion to construe or interpret the provisions of this group insurance policy, to determine eligibility for benefits, and to determine the type and extent of benefits, if any, to be provided. the decisions of the principal in such matters shall be controlling, binding and final as between the principal and persons covered by this group policy, subject to the claims procedures in part iv, section d. article 11 - electronic transactions any transaction relating to this group policy may be conducted by electronic means if performance of the transaction is consistent with applicable state and federal law. any notice required by the provisions of this group policy given by electronic means will have the same force and effect as notice given in writing. this policy has been updated effective january 1, 2014 part ii - policy administration gc 6003 section a - contract,  4'

### 1.2 Generate and Store Embeddings using OpenAI and ChromaDB

In [ ]:
# Import the OpenAI Embedding Function into chroma
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

with open("/content/drive/MyDrive/upgrad/api_keys/OpenAI_key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

# Define the path where chroma collections will be stored
chroma_data_path = '/content/drive/MyDrive/upgrad/Course 3 - RAG/chroma'

# Call PersistentClient()
client = chromadb.PersistentClient(chroma_data_path)

# Set up the embedding function using the OpenAI embedding model
embedding_function = OpenAIEmbeddingFunction(
    api_key=openai.api_key,
    model_name="text-embedding-ada-002"
  )

# Initialise a collection in chroma and pass the embedding_function to it so that it used OpenAI embeddings to embed the documents
insurance_collection = client.get_or_create_collection(
    name='RAG_on_Insurance',
    embedding_function=embedding_function
  )

cache_collection = client.get_or_create_collection(
    name='Insurance_Cache',
    embedding_function=embedding_function
  )

In [ ]:
# Convert the page text and metadata from your dataframe to lists to be able to pass it to chroma
documents_list = insurance_pdfs_data["Page_Text"].tolist()
metadata_list = insurance_pdfs_data['Metadata'].tolist()

# Add the documents and metadata to the collection alongwith generic integer IDs. You can also feed the metadata information as IDs by combining the policy name and page no.
insurance_collection.upsert(
    documents= documents_list,
    ids = [str(i) for i in range(0, len(documents_list))],
    metadatas = metadata_list
)

# Add the documents and metadata to the collection alongwith generic integer IDs. You can also feed the metadata information as IDs by combining the policy name and page no.
# insurance_collection.add(
#     documents= documents_list,
#     ids = [str(i) for i in range(0, len(documents_list))],
#     metadatas = metadata_list
# )

In [ ]:
# Let's take a look at the first few entries in the collection
check = insurance_collection.get(
    ids = ['0'],
    include = ['embeddings', 'documents', 'metadatas']
)

print(f"Embedding Length : {len((check['embeddings'][0]))}")
print(f"Sample Embedding : {(check['embeddings'][0][:5])}...")
print(f"MetaData         : {(check['metadatas'][0])}")
print(f"Page Text        : {(check['documents'][0])}")

Embedding Length : 1536
Sample Embedding : [-0.03006616421043873, 0.02232346124947071, -0.026021866127848625, -0.033365458250045776, -0.005730531178414822]...
MetaData         : {'Page_No.': '1', 'Policy_Name': 'Principal-Sample-Life-Insurance-Policy'}
Page Text        : dorothea glause s655 rhode island john doe 01/01/2014 711 high street george ri 02903 group policy for: rhode island john doe all members group member life insurance print date: 07/16/2014


## <font color = green> 2. Layer 2 : Retrival Layer

### 2.2 Semantic Search with Cache


In [ ]:
def search_db_with_cache(
    query,
    collection = insurance_collection,
    n_results = 10,
    cache_collection = None,
    cache_threshold = 0.2):


    # Set a threshold for cache search
    threshold = cache_threshold

    cache_results = None

    if cache_collection :

      # Searh the Cache collection first
      cache_results = cache_collection.query(
          query_texts=query,
          n_results=1
      )


    if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
          # Query the collection against the user query and return the top 10 results
          results = insurance_collection.query(
          query_texts=query,
          n_results=n_results
          )

          Keys = []
          Values = []

          for key, val in results.items():
            if val is None:
              continue
            for i in range(10):
              Keys.append(str(key)+str(i))
              Values.append(str(val[0][i]))

          cache_collection.add(
              documents= [query],
              ids = [query],
              metadatas = dict(zip(Keys, Values))
          )

          result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
          results_df = pd.DataFrame.from_dict(result_dict)
          results_df

    elif cache_results['distances'][0][0] <= threshold:

          # Initialize empty lists to store the results
          ids = []
          documents = []
          distances = []
          metadatas = []

          cache_result_dict = cache_results['metadatas'][0][0]

          for key, value in cache_result_dict.items():
              if 'ids' in key:
                  ids.append(value)
              elif 'documents' in key:
                  documents.append(value)
              elif 'distances' in key:
                  distances.append(value)
              elif 'metadatas' in key:
                  metadatas.append(value)

          # Create a DataFrame
          results_df = pd.DataFrame({
            'IDs': ids,
            'Documents': documents,
            'Distances': distances,
            'Metadatas': metadatas
          })

    return results_df

In [ ]:
query = input("What is muggling your brain?\n")

search_db_with_cache(
    query=query,
    collection = insurance_collection,
    n_results = 10,
    cache_collection = cache_collection,
    cache_threshold = 0.2)

What is muggling your brain?
How long does the policy last?


,IDs,Documents,Distances,Metadatas
0,21,t he principal may terminate the policyholder'...,0.37746309211796786,"{'Page_No.': 'Page 24', 'Policy_Name': 'Princi..."
1,40,any individual policy issued will then be in f...,0.37746755024492024,"{'Page_No.': 'Page 43', 'Policy_Name': 'Princi..."
2,58,section d - claim procedures article 1 - notic...,0.37859889112396294,"{'Page_No.': 'Page 61', 'Policy_Name': 'Princi..."
3,39,section f - individual purchase rights article...,0.37988388072617424,"{'Page_No.': 'Page 42', 'Policy_Name': 'Princi..."
4,38,i f coverage for a member or dependent termina...,0.38149143011624725,"{'Page_No.': 'Page 41', 'Policy_Name': 'Princi..."
5,34,b. a business assignment; or c. full-time stud...,0.381541293912057,"{'Page_No.': 'Page 37', 'Policy_Name': 'Princi..."
6,33,a member's insurance under this group policy f...,0.38425043459659053,"{'Page_No.': 'Page 36', 'Policy_Name': 'Princi..."
7,14,a. be actively engaged in business for profit ...,0.39167872371909274,"{'Page_No.': 'Page 17', 'Policy_Name': 'Princi..."
8,41,(4) premium will be based on the dependent's a...,0.39254247045237256,"{'Page_No.': 'Page 44', 'Policy_Name': 'Princi..."
9,59,a claimant may request an appeal of a claim de...,0.3942071026238462,"{'Page_No.': 'Page 62', 'Policy_Name': 'Princi..."


### 2.3 Re-Ranking with a Cross Encoder


In [ ]:
# Import the CrossEncoder library from sentence_transformers
from sentence_transformers import CrossEncoder, util
# Initialise the cross encoder model
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
def re_rank(query, results_df):

  # Generate the cross_encoder scores for these pairs
  cross_inputs = [[query, response] for response in results_df['Documents']]
  cross_rerank_scores = cross_encoder.predict(cross_inputs)

  # Store the rerank_scores in results_df
  results_df['Reranked_scores'] = cross_rerank_scores

  # Return the top 3 results after reranking
  results_df = results_df.sort_values(by='Reranked_scores', ascending=False)

  return results_df[["Documents", "Metadatas"]][:3]

In [ ]:

query = input("What is muggling your brain?\n")

results_df = search_db_with_cache(
    query=query,
    collection = insurance_collection,
    n_results = 10,
    cache_collection = cache_collection,
    cache_threshold = 0.4
  )

top_3_RAG = re_rank(
    query,
    results_df
  )

top_3_RAG

What is muggling your brain?
How long does the policy Last? 


,Documents,Metadatas
4,i f coverage for a member or dependent termina...,"{'Page_No.': 'Page 41', 'Policy_Name': 'Princi..."
0,t he principal may terminate the policyholder'...,"{'Page_No.': 'Page 24', 'Policy_Name': 'Princi..."
2,section d - claim procedures article 1 - notic...,"{'Page_No.': 'Page 61', 'Policy_Name': 'Princi..."


## <font color = blue> 3.  Layer 3 : RAG Layer


In [ ]:
def convert_to_dictionary (top_3_RAG) :

  top_3_RAG_dict = {}

  x = 1

  for i in top_3_RAG.values:

    doc = i[0]
    try :
      meta = eval(i[1])
      meta["Policy_Name"] = meta["Policy_Name"].replace('_',' ')
      mets = f'In the page {meta["Page_No."]} of the policy {meta["Policy_Name"]}'
    except :
      meta = i[1]

    top_3_RAG_dict[f'result_{x}'] = {
        'reference' : meta,
        'document_text' : doc
    }
    x+=1

  return top_3_RAG_dict

In [ ]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """

    top_3_RAG_dict = convert_to_dictionary(results_df)

    messages = [
                  {
                    "role": "system",
                    "content": f"""
                                    You are a proficient assistant specializing in the insurance domain, adept at responding to user inquiries concerning insurance policies and related documents."""
                  },
                  {
                    "role": "user",
                    "content": f"""
                                    You are an insurance domain assistant. Read and break down the user query into key elements and sub-questions. Analyze the 'search_results' JSON to find relevant documents and extract pertinent information, including reformatting tables if necessary. Compile the extracted data into a concise, direct response, citing policy names and page numbers from the 'metadata'. Review the response for clarity and completeness, stating if the query is irrelevant if necessary.

                                    ### User Query:
                                    '{query}'.

                                    ### search_results :
                                   '{top_3_RAG_dict}'
                                    The key is the metadata and the value is the document text.

                                    Here is the detailed chain of thought process to follow:

                                    **Step 1: Understand the User Query**
                                    - Carefully read the user query '{query}'.
                                    - Re-write the query with different aspects or sub-questions. Identify key elements such as the type of coverage, conditions, dates, eligibility, and any specific scenarios mentioned.

                                    **Step 2: Analyze the Dataframe for Relevant Information**
                                    - Examine the 'search_results' JSON to identify which documents might contain relevant information.
                                    - For each document in the JSON, scan the text to determine its relevance to the query aspects identified in Step 1.

                                    **Step 3: Extract and Interpret Relevant Information**
                                    - Extract information from the relevant documents that address the different aspects of the query.
                                    - [IMP]**If multiple documents provide relevant details, synthesize the information to form a coherent response**
                                    - Write down the policy exerpt in simple language
                                    - For any tables present in the documents, reformat the data into a clear and understandable table.

                                    **Step 4: Formulate the Response**
                                    - Compile the extracted information into a well-structured and informative answer.
                                    - Ensure the response is concise, direct, and addresses all aspects of the user query.
                                    - Use the 'metadata' to cite the policy names and page numbers for each piece of information used.

                                    **Step 5: Finalize and Cite the Response**
                                    - Review the response to ensure clarity and completeness.
                                    - Include citations for each piece of information, referencing the policy name(s) and page number(s) as obtained from the 'metadata' column.
                                    - If the documents do not contain relevant information, clearly state that the query is irrelevant and provide any guidance for further searches if possible.

                                    Follow the guidelines below when performing the task :
                                      1. Try to provide relevant/accurate numbers if available.
                                      2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                      3. If the document text has tables with relevant information, please reformat the table and return the final information in a tabular in format.
                                      3. Use the Metadatas columns in the dataframe to retrieve and cite the policy name(s) and page numbers(s) as citation.
                                      4. If you can't provide the complete answer, please also provide any information that will help the user to search specific sections in the relevant cited documents.
                                      5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                    """
                  },
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature = 1.2
    )

    return response.choices[0].message.content.split('\n')

In [ ]:
def run_rag(debug=False):
  """
    This function asks for the users query and returns the result to the user
  """
  # Generate the response

  print("The converation with the policy starts now, type", color.BOLD +"\'Exit\'"+color.END , "to",color.BOLD +"stop"+color.END,"\n")

  print(color.BOLD + color.DARKCYAN + "Agent : " + color.END)
  print("What is muggling your brain?\n")
  print(color.BOLD + color.PURPLE + "You: " + color.END)
  query = input()
  print()

  while(query!='Exit'):

    results_df = search_db_with_cache(
        query=query,
        collection = insurance_collection,
        n_results = 10,
        cache_collection = cache_collection,
        cache_threshold = 0.2
      )

    top_3_RAG = re_rank(
        query,
        results_df
      )

    if debug:
      # display(query)
      # display(results_df)
      display(top_3_RAG)

    response = generate_response(query, top_3_RAG)

    print(color.BOLD + color.DARKCYAN + "Agent : " + color.END)
    print("\n".join(response))
    print("\nWhat else is muggling your brain?\n")
    print(color.BOLD + color.PURPLE + "You: " + color.END)
    query = input()
    print()


In [ ]:
run_rag()

The converation with the policy starts now, type 'Exit' to stop 

Agent : 
What is muggling your brain?

You: 
How many years does the policy cover my life for?

Agent : 

**Response:**

Based on the search results, the "Principal-Sample-Life-Insurance-Policy" document provides relevant information regarding the coverage duration of the policy for both members and dependents. Here is a concise summary:

- For a member, individual purchase rights allow the conversion of group coverage into an individual policy for life insurance only. The policy does not include disability or other benefits.
- A member qualifies for individual purchase if their insurance under the group policy terminates after being continuously insured for at least five years.
- An individual policy must be applied for within 31 days after the termination date of the member's life insurance.
- The policy includes age-based premiums and is updated effective January 1, 2014.

For more detailed information and specific du

## <font color = Purple> 4. Three Question Test



> **What are the policy coverage details for accidents involving public transport?**

    Interpretation Needed: Understanding the specific conditions under which coverage can be continued and recognizing the distinctions made for different reasons for being outside the United States.



> **How does the policy address claims related to natural disasters?**

    Interpretation Needed: Analyzing the policy's provisions for handling fraudulent claims and understanding the steps taken by the insurer to terminate coverage based on fraudulent activities

> **What benefits are available if the insured is permanently disabled in an accident?**

    Interpretation Needed: Interpreting the requirements for continuation of coverage for a dependent child with a developmental disability or physical handicap, and identifying the necessary documentation and proof that must be submitted

In [111]:
# run_rag()

#### Question 1

In [112]:
run_rag(debug=True)

The converation with the policy starts now, type 'Exit' to stop 

Agent : 
What is muggling your brain?

You: 
What are the policy coverage details for accidents involving public transport?



,Documents,Metadatas
0,exposure exposure to the elements will be pres...,"{'Page_No.': '55', 'Policy_Name': 'Principal-S..."
1,"f . claim requirements listed in part iv, sect...","{'Page_No.': '54', 'Policy_Name': 'Principal-S..."
7,% of scheduled covered loss benefit loss of sp...,"{'Page_No.': '57', 'Policy_Name': 'Principal-S..."


Agent : 

### Response:
#### Policy Coverage details for Accidents Involving Public Transport:

Based on the search results provided in the documents indexed, the relevant policy information can be found in the **Principal-Sample-Life-Insurance-Policy** document on page 55, page 54, and page 57. Below are the extracted details related to accidents involving public transport from the respective pages:

1. **Loss Due to Accidental Bodily Injury:**
   - If a member incurs a loss within 365 days as a result of exposure due to accidental bodily injury, covered by the group policy, the benefit will be provided. (*Principal-Sample-Life-Insurance-Policy*, Page 55)

2. **Seat Belt/Airbag Benefit for Accidental Injuries in Automobiles:**
   - An additional benefit of $10,000 will be paid if a member loses their life in an automobile accident while using a factory-installed seat belt or proper airbag. (*Principal-Sample-Life-Insurance-Policy*, Page 55)

3. **Benefits Payable for Losses:**
   - De

In [113]:
run_rag(debug=False)

The converation with the policy starts now, type 'Exit' to stop 

Agent : 
What is muggling your brain?

You: 
What are the policy coverage details for accidents involving public transport?

Agent : 

### Response:
Based on the search results, I found relevant information from the policy named 'Principal-Sample-Life-Insurance-Policy' on pages 55, 54, and 57 that may address the policy coverage details for accidents involving public transport.

#### Policy Coverage Details for Accidents Involving Public Transport:

1. **Seat Belt/Airbag Benefit:**
   - If a member loses their life due to an accidental injury sustained while driving or riding in an automobile, an additional benefit of $10,000 will be paid to the beneficiary named for member life insurance.
   - Conditions:
     - The automobile must be equipped with factory-installed seat belts.
     - The seat belt must have been in actual use by the member and properly fastened at the time of the accident.
     - The position of the se

#### Question 2 :

In [114]:
run_rag(debug=True)

The converation with the policy starts now, type 'Exit' to stop 

Agent : 
What is muggling your brain?

You: 
How does the policy address claims related to natural disasters?



,Documents,Metadatas
0,"f . claim requirements listed in part iv, sect...","{'Page_No.': '54', 'Policy_Name': 'Principal-S..."
1,section d - claim procedures article 1 - notic...,"{'Page_No.': '61', 'Policy_Name': 'Principal-S..."
2,exposure exposure to the elements will be pres...,"{'Page_No.': '55', 'Policy_Name': 'Principal-S..."


Agent : 

### Response:

Based on the search results, the policy that addresses claims related to natural disasters is the **Principal-Sample-Life-Insurance-Policy**.

Here are the key excerpts from the policy related to claims and benefits concerning natural disasters:

| Article/Section | Description |
|-----------------|---------------|
| Article 3 - Benefits Payable | - % of scheduled benefit for various losses <br> - Provisions for loss of life, severing of hands/feet, loss of sight, and multiple losses due to the same accident |
| Article 4 - Seat Belt/Airbag Benefit | - Additional benefit of $10,000 if a member loses their life in an automobile accident with specific conditions on seat belt and airbag usage |

These details can be found on:
- Page 54 for the benefits payable section under **Principal-Sample-Life-Insurance-Policy**
- Page 55 for the exposure section under **Principal-Sample-Life-Insurance-Policy**

For more detailed information and specific claim procedures regar

In [116]:
run_rag(debug=False)

The converation with the policy starts now, type 'Exit' to stop 

Agent : 
What is muggling your brain?

You: 
How does the policy address claims related to natural disasters?

Agent : 
### Response:

#### Policy: Principal-Sample-Life-Insurance-Policy
**Page No.: 54**
- The policy addresses claims related to natural disasters based on specific qualifications and benefits. 
- If the loss is due to a natural disaster resulting in specific injuries, the principal will pay varying percentages of the scheduled benefit amount depending on the severity of the injury.
  
#### Explaining in Simple Terms:
When a natural disaster causes injuries as per terms in the policy from the Principal-Sample-Life-Insurance-Policy, the insurance becomes payable based on the degree of injury suffered. For example, if there is a loss of life or limb due to a natural disaster, the insurance will cover a percentage of the agreed benefit amount.

| Type of Loss or Disappearance      | Benefit Covered            

#### Question 3 :

In [117]:
run_rag(debug=True)

The converation with the policy starts now, type 'Exit' to stop 

Agent : 
What is muggling your brain?

You: 
What benefits are available if the insured is permanently disabled in an accident?



,Documents,Metadatas
1,payment of benefits will be subject to the ben...,"{'Page_No.': '49', 'Policy_Name': 'Principal-S..."
5,"f . claim requirements listed in part iv, sect...","{'Page_No.': '54', 'Policy_Name': 'Principal-S..."
2,exposure exposure to the elements will be pres...,"{'Page_No.': '55', 'Policy_Name': 'Principal-S..."


Agent : 
**User Query Breakdown:**
- Query: What benefits are available if the insured is permanently disabled in an accident?
- Key Elements: 
    - Type of coverage available for permanent disability
    - Eligibility criteria for receiving benefits
    - Probable percentage or amount covered for different disability scenarios

**Analysis of search_results:**
- Relevant document found:
    - Policy Name: Principal-Sample-Life-Insurance-Policy
    - Page No.: 49, 54, 55

**Extracted Information from Principal-Sample-Life-Insurance-Policy:**

1. **Benefit for Permanent Disability:**
    - **Policy**: Principal-Sample-Life-Insurance-Policy
    - **Page No.**: 49
    - **Excerpt**: Insured must qualify for coverage during disability by being ADL disabled prior to age 60, continuously disabled, under a physician's care, and providing necessary proof of disability. Additionally, written proof of ADL disability must be submitted within one year of its onset.

2. **Benefits for Different Dis

In [ ]:
run_rag(debug=False)

The converation with the policy starts now, type 'Exit' to stop 

Agent : 
What is muggling your brain?

You: 
How does the policy address claims related to natural disasters?

Agent : 
### Response:

#### Policy: Principal-Sample-Life-Insurance-Policy
**Page No.: 54**
- The policy addresses claims related to natural disasters based on specific qualifications and benefits. 
- If the loss is due to a natural disaster resulting in specific injuries, the principal will pay varying percentages of the scheduled benefit amount depending on the severity of the injury.
  
#### Explaining in Simple Terms:
When a natural disaster causes injuries as per terms in the policy from the Principal-Sample-Life-Insurance-Policy, the insurance becomes payable based on the degree of injury suffered. For example, if there is a loss of life or limb due to a natural disaster, the insurance will cover a percentage of the agreed benefit amount.

| Type of Loss or Disappearance      | Benefit Covered            